Utworzyć ramkę danych opisującą w jaki sposób dany lek wchodzi w interakcje ze swoimi targetami.

In [12]:
import pandas as pd
import lxml
import xml.etree.ElementTree as ET
import networkx as nx
import matplotlib.pyplot as plt
from pprint import pprint

In [13]:
path = 'data/drugbank_partial.xml'
ns = {'db': 'http://www.drugbank.ca'}

In [14]:
tree = ET.parse(path)
root = tree.getroot()

In [15]:
def unwrap(field, node: ET.Element):
    return getattr(node.find(f"db:{field}", ns), "text", None)

In [16]:
def unwrap_attrib(attrib, node: ET.Element):
    return node.attrib.get(attrib)

In [22]:
def target_interactions_df(drug_id):
    drug = root.find(f"db:drug[db:drugbank-id='{drug_id}']", ns)    
    data = []
    index = []
    for target in drug.findall('db:targets/db:target', ns):
        polypeptide = target.find('db:polypeptide', ns)
        index.append(unwrap("id", target))
        data.append({
            "Target": unwrap("name", target),
            "Organism": unwrap("organism", target),
            "Actions": [a.text for a in target.findall('db:actions/db:action', ns)],
            "Known action": unwrap("known-action", target),
            "General Function": unwrap("general-function", polypeptide),
            "Specific Function": unwrap("specific-function", polypeptide),
        })

    return pd.DataFrame(data, index=index)

In [23]:
target_interactions_df("DB00002")

,Target,Organism,Actions,Known action,General Function,Specific Function
BE0000767,Epidermal growth factor receptor,Humans,[binder],yes,Ubiquitin protein ligase binding,Receptor tyrosine kinase binding ligands of th...
BE0000901,Low affinity immunoglobulin gamma Fc region re...,Humans,[binder],unknown,None,Receptor for the Fc region of immunoglobulins ...
BE0002094,Complement C1q subcomponent subunit A,Humans,[binder],unknown,None,C1q associates with the proenzymes C1r and C1s...
BE0002095,Complement C1q subcomponent subunit B,Humans,[binder],unknown,None,C1q associates with the proenzymes C1r and C1s...
BE0002096,Complement C1q subcomponent subunit C,Humans,[binder],unknown,None,C1q associates with the proenzymes C1r and C1s...
BE0002097,Low affinity immunoglobulin gamma Fc region re...,Humans,[binder],unknown,None,Receptor for the Fc region of IgG. Binds compl...
BE0000710,High affinity immunoglobulin gamma Fc receptor I,Humans,[binder],unknown,Receptor signaling protein activity,High affinity receptor for the Fc region of im...
BE0002098,Low affinity immunoglobulin gamma Fc region re...,Humans,[binder],unknown,None,Binds to the Fc region of immunoglobulins gamm...
